In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,\
Conv2D, MaxPooling2D
import skimage
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from skimage.io import imread
from skimage.transform import resize

Using Theano backend.


In [2]:
train_path = './nuts_n_bolts/train'
test_path = './nuts_n_bolts/test'
val_path = './nuts_n_bolts/val'

In [3]:
# Construct a model in keras for nuts_and_bolts classfication
# The model consists of two convolutional layers followed by Relu
# 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50,65,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [4]:
batch_size = 25
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=(50, 65),  # all images will be resized to 150x150
        batch_size=batch_size,
        classes = ['nuts','bolts'],
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 48 images belonging to 2 classes.


In [5]:
batch_size = 2
val_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_generator = train_datagen.flow_from_directory(
        val_path,  # this is the target directory
        target_size=(50, 65),  # all images will be resized to 50x65
        batch_size=batch_size,
        classes = ['nuts','bolts'],
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 9 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=100 // batch_size)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '9116' (I am process '9900')


Epoch 1/5
50/50 [==============================] - 28s - loss: 0.6306 - acc: 0.7653 - val_loss: 0.4322 - val_acc: 0.8222
Epoch 2/5
50/50 [==============================] - 27s - loss: 0.1564 - acc: 0.9394 - val_loss: 0.7469 - val_acc: 0.7000
Epoch 3/5
50/50 [==============================] - 27s - loss: 0.0880 - acc: 0.9725 - val_loss: 0.8873 - val_acc: 0.6889
Epoch 4/5
16/50 [========>.....................] - ETA: 17s - loss: 0.1093 - acc: 0.9633

In [8]:
bolt_path = './nuts_n_bolts/test/bolts/'
nut_path = './nuts_n_bolts/test/nuts'
image_paths = [os.path.join(bolt_path, img_name) for img_name in os.listdir(bolt_path)]
image_paths += [os.path.join(nut_path, img_name) for img_name in os.listdir(nut_path)]
test_images = np.zeros((len(image_paths),50,65,3))

for ind,image_name in enumerate(image_paths):
    test_images[ind, :, :, :] = resize(imread(image_name), (50,65,3))
    
prediction = model.predict(test_images)
labels = ['nuts' if img_pred[0] > img_pred[1] else 'bolts' for img_pred in prediction]
image_names = [os.path.basename(path) for path in image_paths]
matched_prediction = list(zip(image_names, labels))
print(matched_prediction)

[('bolt_1.jpg', 'nuts'), ('bolt_3.jpg', 'bolts'), ('bolt_4.jpg', 'bolts'), ('bolt_2.jpg', 'bolts'), ('nut_1.jpg', 'nuts'), ('nut_2.jpg', 'nuts'), ('nut_3.jpg', 'nuts'), ('nut_4.jpg', 'nuts')]


/home/senthil/.virtualenvs/anthill/lib/python3.4/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
